In [1]:
import tensorflow as tf
import os 
import numpy as np
import pandas as pd
from model import CNN_Model,Params
from Env import Env
from time import sleep

C:\Users\Armughan.Shahid\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
class experience_replay_buffer:
#     def __init__(self,size,dtypes):
#         self.column_names=['state','action','next_state','reward','done']
#         self.buffer={self.column_names[i]:np.empty(size,dtype=dtypes[i]) for i in np.arange(len(self.column_names)) }
#         self.num_items=0
#         self.capacity=size
    def __init__(self,buffer_len,sample):
        self.column_names=['state','action','next_state','reward','done']
        self.buffer={col_name:np.empty(shape=[buffer_len,*np.array(item).shape],dtype=np.array(item).dtype) for col_name,item in zip(self.column_names,sample) }
        self.num_items=0
        self.capacity=buffer_len
        self.add_ind=0
    def add_experience(self,state,action,next_state,reward,done):
        ind=self.add_ind
        self.add_ind=(self.add_ind+1)%self.capacity
#         ind=self.num_items
        if self.num_items<self.capacity:
            self.num_items+=1
#         else:
#             ind=np.random.randint(low=0,high=self.capacity,size=1,dtype=np.int32)
            
        self.buffer['state'][ind]=state
        self.buffer['action'][ind]=action
        self.buffer['next_state'][ind]=next_state
        self.buffer['reward'][ind]=reward
        self.buffer['done'][ind]=done
           
    
    def get_batch(self,batch_size):
        inds=np.random.randint(low=0,high=self.num_items,size=batch_size,dtype=np.int32)
        return self.buffer['state'][inds],self.buffer['action'][inds],self.buffer['next_state'][inds],self.buffer['reward'][inds],self.buffer['done'][inds]
    

In [3]:
class Q_Network(CNN_Model):
    def __init__(self,max_experience_buffer_len=120,param_dict={},restore_params=False,pickle_file_path=""):
        CNN_Model.__init__(self,param_dict,restore_params,pickle_file_path)
        self.max_experience_buffer_len=max_experience_buffer_len
        
    def form_loss(self,logits,targets):
        entropies=self.params.loss_fn(labels=targets,logits=logits)
        return entropies
        
    def Build_model(self):
        self.build_model_till_logits()
        with tf.variable_scope(self.params.name_scope):
            #logits are q values]
            self.max_q_value_actions=tf.squeeze(tf.argmax(self.logits,axis=1)) #value which has the highest q value
            self.max_q_values=tf.reduce_max(self.logits,axis=1)
            
            #placeholder for action at current timestep
            self.actions=self.form_placeholder((None),tf.int32)
            one_hot_actions=tf.one_hot(indices=self.actions,depth=self.params.num_outputs)
            q_vals=tf.reduce_sum(self.logits*one_hot_actions,axis=1)
            
            
            
            #placeholder for max next state q values,rewards and discount rate
            self.max_q_values_next_state=self.form_placeholder((None),tf.float32)
            self.rewards=self.form_placeholder((None),tf.float32)
            self.notended=self.form_placeholder((None),tf.float32)
            self.discount_rate=self.form_placeholder([],tf.float32)
            
            self.loss=tf.reduce_mean(tf.square((self.rewards+(self.discount_rate*(self.max_q_values_next_state*self.notended)))-q_vals))
#             self.qvalues_next=self.max_q_values_next_state*self.notended
#             self.discounted_qvalues_next=self.discount_rate*self.qvalues_next
#             self.targets=self.rewards+self.discounted_qvalues_next
#             self.diff=self.targets-q_vals
#             self.squared_diff=tf.square(self.diff)
#             self.loss=tf.reduce_mean(self.squared_diff)
        
            #computing gradients 
            optimizer=self.params.optimizer_fn(learning_rate=self.lr_placeholder)
            self.grads_and_vars=optimizer.compute_gradients(loss=self.loss,var_list=self.model_trainable_variables)
            
            self.train_op=optimizer.apply_gradients(grads_and_vars=self.grads_and_vars,global_step=self.step_no)
            self.model_variables=tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=self.params.name_scope)
            self.saver=tf.train.Saver(var_list=self.model_variables)

            self.initializer=tf.global_variables_initializer()
    
    def add_to_experience_replay(self,state,action,next_state,reward,done):
        
        if not hasattr(self,"experience_replay_buffer"):
            sample=[state,action,next_state,reward,done]
            self.experience_replay_buffer=experience_replay_buffer(buffer_len=self.max_experience_buffer_len,sample=sample)
        self.experience_replay_buffer.add_experience(state,action,next_state,reward,done)



    def train(self,sess,episodes,steps,initial_epsilon,final_epsilon,epsilon_dec,train_t,discount_rate,batch_size,env,save_dir,save_every_n_iter,log_every_n_iter,initialize=False,set_logging=True,num_frames_to_repeat_action=4):
        if initialize:
            print ("Initializing.....\n")
            sess.run([self.initializer])
        if set_logging:
            print ("Setting up for Logging ...\n")
            log_dir,set_logging=self.create_log_directory_if_doesnt_exist(save_dir)
        if set_logging: #creating file handlers if dir cretaed or found in above statement
            print("Logging called but no code implemented")
#                 train_writer = tf.summary.FileWriter(os.path.join(log_dir,'train'), sess.graph)
#                 validation_writer = tf.summary.FileWriter(os.path.join(log_dir ,'validation'))
        print ("Retreiveing step no...\n")
        [iter_no]=sess.run([self.step_no]) 
        epsilon=initial_epsilon
        for episode in np.arange(episodes):
            state=env.reset()
            step=0
            episode_reward=0
            episode_loss=0
            previous_action=None
            for step in np.arange(steps):
                #choosing action 
#                 action=-1
                if step%num_frames_to_repeat_action==0:
                    if epsilon>final_epsilon and iter_no>train_t:
                            epsilon-=epsilon_dec

    #                 if  iter_no<train_t or (np.random.random(1)<epsilon):
                    if  (np.random.random(1)<epsilon):
                        action=np.random.randint(low=0,high=self.params.num_outputs,size=1,dtype=np.int32)
                    else:
                        feed_dict={self.X:np.expand_dims(state,axis=0),self.lr_placeholder:self.params.learning_rate,self.training_mode:True}
                        [action]=sess.run([self.max_q_value_actions],feed_dict=feed_dict)
                    action=np.squeeze(action)
                else:
                    action=previous_action
#                 print(action)
                next_state,reward,done,info=env.step(action)
                episode_reward+=reward
                self.add_to_experience_replay(state,action,next_state,reward,done)
                previous_action=action
                episode_has_finished=done

                state=next_state

                if self.experience_replay_buffer.num_items>train_t: #perform training if there are enough experiences
#                     print("buffer filled")
                    
                    
                    #performing training step
                    states,actions,next_states,rewards,dones=self.experience_replay_buffer.get_batch(batch_size=batch_size)
                    

                    #finding vals of next states
#                     print (next_states.shape)
                    feed_dict={self.X:next_states,self.lr_placeholder:self.params.learning_rate,self.training_mode:True}
                    [max_q_vals_next_state]=sess.run([self.max_q_values],feed_dict=feed_dict)
#                     max_q_vals_next_state[nan_inds]=0

                    #calculating loss and running train op
#                     print("shapes\n")
#                     print(states.shape)
#                     print(actions.shape)
#                     print(max_q_vals_next_state.shape)
#                     print(rewards.shape)
#                     print(dones.shape)
#                     print(discount_rate)
                    
#                     print("li\n")
#                     print(states)
#                     print(actions)
#                     print(max_q_vals_next_state)
#                     print(rewards)
#                     print(dones)
#                     print(discount_rate)
#                     self.qvalues_next=self.max_q_values_next_state*self.notended
#             self.discounted_qvalues_next=self.discount_rate*qvalues_next
#             self.targets=self.rewards+discounted_qvalues_next
#             self.diff=targets-q_vals
#             self.squared_diff=tf.square(diff)
#             self.loss=tf.reduce_mean(squared_diff)
                    feed_dict={self.X:states,self.actions:actions,self.max_q_values_next_state:max_q_vals_next_state,self.rewards:rewards,self.notended:((np.logical_not(dones)).astype(np.int32)),self.discount_rate:discount_rate,self.lr_placeholder:self.params.learning_rate,self.training_mode:True}
                    loss,_=sess.run([self.loss,self.train_op],feed_dict=feed_dict)
                    episode_loss+=loss
                    iter_no+=1
                    if (iter_no)%save_every_n_iter==0:
                        print("^^^^ saving model ^^^^ \n")
                        self.save_model(sess,save_dir,self.step_no)
                    if (iter_no)%log_every_n_iter==0:
                        print ("Trainaing Step:\t Iteration no={} Game Step ={} loss={} ".format(iter_no,step,loss))
                if episode_has_finished:
                    break
            print ("===================>Episode {} Ended <===================\n".format(episode)) 
            print ("=======>\t Episode Length={} \t<=======\n".format(step))   
            print ("=======>\t Episode Reward={} \t<=======\n".format(episode_reward))
            print ("=======>\t Mean Episode Loss={} \t<=======\n".format(episode_loss/step))
            
    def test(self,initialize,env,sleep_time=0.5):
        if initialize:
            print ("Initializing.....\n")
            sess.run([self.initializer])
            
        state=env.reset()
        done=False
        while not done:
            feed_dict={self.X:np.expand_dims(state,axis=0),self.lr_placeholder:self.params.learning_rate,self.training_mode:False}
            [action]=sess.run([self.max_q_value_actions],feed_dict=feed_dict)
            action=np.squeeze(action)
            state,reward,done,info=env.step(action)
            env.render()
            sleep(sleep_time)
        env.close()

In [4]:
env=Env('SpaceInvaders-v0',convert_to_grayscale=True,crop=True,valid_Y=[20,-10],valid_X=[10,-10],resize=True,resize_Y=90,resize_X=70,normalize=True,num_of_frames_per_stack=4)
params={
    'input_shape':[None, *env.image_shape],
    'num_outputs':env.action_space,
    
    'layer_hierarchy':[
        {'layer_type':'conv_layer','kernel_size':8,'kernel_strides':4,'num_filters':32,'padding':'valid'},
#         {'layer_type':'batch_normalization_layer'},
        {'layer_type':'activation_layer'},
        {'layer_type':'conv_layer','kernel_size':4,'kernel_strides':2,'num_filters':64,'padding':'valid'},
#         {'layer_type':'batch_normalization_layer'},
        {'layer_type':'activation_layer'},
        {'layer_type':'conv_layer','kernel_size':3,'kernel_strides':2,'num_filters':64,'padding':'valid'},
#         {'layer_type':'batch_normalization_layer'},
        {'layer_type':'activation_layer'},
        {'layer_type':'flattening_layer'},
        {'layer_type':'fc_layer','num_hidden_units':512},
#         {'layer_type':'batch_normalization_layer'},
        {'layer_type':'activation_layer'}
#         {'layer_type':'dropout_layer','dropout_probability':0.2},
#         {'layer_type':'fc_layer','num_hidden_units':50},
# #         {'layer_type':'batch_normalization_layer'},
#         {'layer_type':'activation_layer'}
#         {'layer_type':'dropout_layer','dropout_probability':0.2}
        
    ],
    'initializer_fn':tf.contrib.layers.variance_scaling_initializer,
    'activation_fn':tf.nn.elu,
#     'loss_fn':tf.nn.sparse_softmax_cross_entropy_with_logits, #carefull
    'learning_rate':0.001,
    'optimizer_fn':tf.train.AdamOptimizer,
    'logdir':'/tf_logs_rnn/run/',
    'name_scope':'q_network_with_frames'
}
print (params['num_outputs'])

c:\users\armughan.shahid\gym\gym\__init__.py:22: UserWarning: DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.
  warnings.warn('DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.')


Resetting Environment...

6


In [6]:
n_episodes=50
max_steps=50000
save_every_n_iter=50
log_every_n_iter=50
initialize=False
save_dir="deep_q_saves"
max_experience_buffer_len=10000
initial_epsilon=0.5#1
final_epsilon=0.0001
epsilon_dec=0.00001
train_t=1000
discount_rate=0.9
batch_size=120

tf.reset_default_graph()

    

model=""
with tf.Session() as sess:
    params['input_shape']
    if(not initialize):
        model=Q_Network(max_experience_buffer_len,params,restore_params=True,pickle_file_path="deep_q_saves/q_network_with_frames/model_object.pkl")
        model.Build_model()
        model.restore_model(sess,save_dir)
        model.params.learning_rate=0.001
    else:
        model=Q_Network(max_experience_buffer_len,params,restore_params=False,pickle_file_path="deep_q_saves/q_network_with_frames/model_object.pkl")
        model.Build_model()
    
    model.train(sess=sess,episodes=n_episodes,steps=max_steps,initial_epsilon=initial_epsilon,final_epsilon=final_epsilon,epsilon_dec=epsilon_dec,train_t=train_t,discount_rate=discount_rate,batch_size=batch_size,env=env,save_dir=save_dir,save_every_n_iter=save_every_n_iter,log_every_n_iter=log_every_n_iter,initialize=initialize,set_logging=True)
#     model.test(initialize=True,env=env)
    env.close()

restoring path:D:\dino_game_current\AI_Plays_Dino_Game\deep_q_saves\q_network_with_frames
INFO:tensorflow:Restoring parameters from D:\dino_game_current\AI_Plays_Dino_Game\deep_q_saves\q_network_with_frames\model_weights.ckpt-81800
Setting up for Logging ...

Logging called but no code implemented
Retreiveing step no...

Resetting Environment...

^^^^ saving model ^^^^ 

Trainaing Step:	 Iteration no=81850.0 Game Step =1049 loss=5.992928981781006 
===================>Episode 0 Ended <===================

=======>	 Episode Length=1051 	<=======

=======>	 Episode Reward=270.0 	<=======

=======>	 Mean Episode Loss=0.3245964736512 	<=======

Resetting Environment...

^^^^ saving model ^^^^ 

Trainaing Step:	 Iteration no=81900.0 Game Step =47 loss=1.0200822353363037 
^^^^ saving model ^^^^ 

Trainaing Step:	 Iteration no=81950.0 Game Step =97 loss=2.583500623703003 
^^^^ saving model ^^^^ 

Trainaing Step:	 Iteration no=82000.0 Game Step =147 loss=1.9424474239349365 
^^^^ saving model ^^

^^^^ saving model ^^^^ 

Trainaing Step:	 Iteration no=85250.0 Game Step =400 loss=7.496454238891602 
^^^^ saving model ^^^^ 

Trainaing Step:	 Iteration no=85300.0 Game Step =450 loss=6.594360828399658 
^^^^ saving model ^^^^ 

Trainaing Step:	 Iteration no=85350.0 Game Step =500 loss=3.1520955562591553 
^^^^ saving model ^^^^ 

Trainaing Step:	 Iteration no=85400.0 Game Step =550 loss=3.5137531757354736 
^^^^ saving model ^^^^ 

Trainaing Step:	 Iteration no=85450.0 Game Step =600 loss=5.39811372756958 
^^^^ saving model ^^^^ 

Trainaing Step:	 Iteration no=85500.0 Game Step =650 loss=4.614497184753418 
===================>Episode 4 Ended <===================

=======>	 Episode Length=654 	<=======

=======>	 Episode Reward=60.0 	<=======

=======>	 Mean Episode Loss=9.695361201187158 	<=======

Resetting Environment...

^^^^ saving model ^^^^ 

Trainaing Step:	 Iteration no=85550.0 Game Step =45 loss=3.140728712081909 
^^^^ saving model ^^^^ 

Trainaing Step:	 Iteration no=85600.0 G

^^^^ saving model ^^^^ 

Trainaing Step:	 Iteration no=88750.0 Game Step =347 loss=1.1617988348007202 
^^^^ saving model ^^^^ 

Trainaing Step:	 Iteration no=88800.0 Game Step =397 loss=5.506714344024658 
^^^^ saving model ^^^^ 

Trainaing Step:	 Iteration no=88850.0 Game Step =447 loss=2.4468798637390137 
^^^^ saving model ^^^^ 

Trainaing Step:	 Iteration no=88900.0 Game Step =497 loss=2.317195415496826 
^^^^ saving model ^^^^ 

Trainaing Step:	 Iteration no=88950.0 Game Step =547 loss=4.671149730682373 
^^^^ saving model ^^^^ 

Trainaing Step:	 Iteration no=89000.0 Game Step =597 loss=1.4049108028411865 
^^^^ saving model ^^^^ 

Trainaing Step:	 Iteration no=89050.0 Game Step =647 loss=2.3621108531951904 
^^^^ saving model ^^^^ 

Trainaing Step:	 Iteration no=89100.0 Game Step =697 loss=2.757065773010254 
===================>Episode 9 Ended <===================

=======>	 Episode Length=712 	<=======

=======>	 Episode Reward=130.0 	<=======

=======>	 Mean Episode Loss=2.6617948695

Trainaing Step:	 Iteration no=92300.0 Game Step =466 loss=3.0655643939971924 
===================>Episode 13 Ended <===================

=======>	 Episode Length=507 	<=======

=======>	 Episode Reward=10.0 	<=======

=======>	 Mean Episode Loss=5.576646208763123 	<=======

Resetting Environment...

^^^^ saving model ^^^^ 

Trainaing Step:	 Iteration no=92350.0 Game Step =8 loss=4.5837297439575195 
^^^^ saving model ^^^^ 

Trainaing Step:	 Iteration no=92400.0 Game Step =58 loss=5.183407306671143 
^^^^ saving model ^^^^ 

Trainaing Step:	 Iteration no=92450.0 Game Step =108 loss=2.6208388805389404 
^^^^ saving model ^^^^ 

Trainaing Step:	 Iteration no=92500.0 Game Step =158 loss=4.46624755859375 
^^^^ saving model ^^^^ 

Trainaing Step:	 Iteration no=92550.0 Game Step =208 loss=4.587515354156494 
^^^^ saving model ^^^^ 

Trainaing Step:	 Iteration no=92600.0 Game Step =258 loss=4.691145896911621 
^^^^ saving model ^^^^ 

Trainaing Step:	 Iteration no=92650.0 Game Step =308 loss=11.196

===================>Episode 17 Ended <===================

=======>	 Episode Length=1082 	<=======

=======>	 Episode Reward=275.0 	<=======

=======>	 Mean Episode Loss=6.671104383171156 	<=======

Resetting Environment...

^^^^ saving model ^^^^ 

Trainaing Step:	 Iteration no=95900.0 Game Step =38 loss=6.548985958099365 
^^^^ saving model ^^^^ 

Trainaing Step:	 Iteration no=95950.0 Game Step =88 loss=3.83906626701355 
^^^^ saving model ^^^^ 

Trainaing Step:	 Iteration no=96000.0 Game Step =138 loss=7.495528697967529 
^^^^ saving model ^^^^ 

Trainaing Step:	 Iteration no=96050.0 Game Step =188 loss=5.22810697555542 
^^^^ saving model ^^^^ 

Trainaing Step:	 Iteration no=96100.0 Game Step =238 loss=3.106646776199341 
^^^^ saving model ^^^^ 

Trainaing Step:	 Iteration no=96150.0 Game Step =288 loss=5.715848922729492 
^^^^ saving model ^^^^ 

Trainaing Step:	 Iteration no=96200.0 Game Step =338 loss=4.385396480560303 
^^^^ saving model ^^^^ 

Trainaing Step:	 Iteration no=96250.0 Ga

Trainaing Step:	 Iteration no=99250.0 Game Step =289 loss=1.8190727233886719 
^^^^ saving model ^^^^ 

Trainaing Step:	 Iteration no=99300.0 Game Step =339 loss=1.954352855682373 
^^^^ saving model ^^^^ 

Trainaing Step:	 Iteration no=99350.0 Game Step =389 loss=5.552530288696289 
^^^^ saving model ^^^^ 

Trainaing Step:	 Iteration no=99400.0 Game Step =439 loss=4.9918599128723145 
^^^^ saving model ^^^^ 

Trainaing Step:	 Iteration no=99450.0 Game Step =489 loss=2.945768117904663 
^^^^ saving model ^^^^ 

Trainaing Step:	 Iteration no=99500.0 Game Step =539 loss=3.7758219242095947 
===================>Episode 23 Ended <===================

=======>	 Episode Length=582 	<=======

=======>	 Episode Reward=135.0 	<=======

=======>	 Mean Episode Loss=2.853358414164933 	<=======

Resetting Environment...

^^^^ saving model ^^^^ 

Trainaing Step:	 Iteration no=99550.0 Game Step =6 loss=1.214494228363037 
^^^^ saving model ^^^^ 

Trainaing Step:	 Iteration no=99600.0 Game Step =56 loss=1.34

KeyboardInterrupt: 

In [6]:
# # a=np.arange(6).reshape(3,2)
# # b=np.empty(6,type(a))
# # print (b)
# # b[1]=a
# # print (b.shape)
# # print(type(b))
# # np.isnan(np.array(b))
# a=np.array([[1,2],[3,4]])
# # a=np.array(a)
# print (1,*a.shape)
# print (a.dtype)
# a=np.array([6])
# print (a)
# print (np.squeeze(a))

In [8]:
n_episodes=50
max_steps=50000
save_every_n_iter=20
log_every_n_iter=50
initialize=False
save_dir="deep_q_saves"
max_experience_buffer_len=10000
initial_epsilon=1
final_epsilon=0.0001
epsilon_dec=0.00001
train_t=1000
discount_rate=0.9
batch_size=120

tf.reset_default_graph()

    

model=""
with tf.Session() as sess:
    params['input_shape']
    if(not initialize):
        model=Q_Network(max_experience_buffer_len,params,restore_params=True,pickle_file_path="deep_q_saves/q_network_with_frames/model_object.pkl")
        model.Build_model()
        model.restore_model(sess,save_dir)
        model.params.learning_rate=0.0005
    else:
        model=Q_Network(max_experience_buffer_len,params,restore_params=False,pickle_file_path="deep_q_saves/q_network_with_frames/model_object.pkl")
        model.Build_model()
    
#     model.train(sess=sess,episodes=n_episodes,steps=max_steps,initial_epsilon=initial_epsilon,final_epsilon=final_epsilon,epsilon_dec=epsilon_dec,train_t=train_t,discount_rate=discount_rate,batch_size=batch_size,env=env,save_dir=save_dir,save_every_n_iter=save_every_n_iter,log_every_n_iter=log_every_n_iter,initialize=initialize,set_logging=True)
    model.test(initialize=True,env=env,sleep_time=0.05)

restoring path:D:\dino_game_current\AI_Plays_Dino_Game\deep_q_saves\q_network_with_frames
INFO:tensorflow:Restoring parameters from D:\dino_game_current\AI_Plays_Dino_Game\deep_q_saves\q_network_with_frames\model_weights.ckpt-100050
Initializing.....

Resetting Environment...

